In [47]:
# DB Instance Values
DB_INSTANCE_NAME = "jerry-v1-db-instance"
DB_VERSION = "POSTGRES_17"
CPU = "1"
MEMORY = "4gb"
REGION = "us-central1"
EDITION = "enterprise"
ROOT_PASS = "gamalziplockshisfartstomarinateinthefridgebeforereleasingthemintheoffice"


# DB Values
PROJECT_NAME = "ari-dev-463216"
DB_NAME = "jerry_v1_extracted_db"
PG_DRIVER = "pg8000"
USER = "postgres"
PASSWORD = "gamalziplockshisfartstomarinateinthefridgebeforereleasingthemintheoffice"
CONNECTION_NAME = f"{PROJECT_NAME}:{REGION}:{DB_INSTANCE_NAME}"


# Bucket Values
BUCKET_BASE_URL = "gs://jerry_v1_upload/"


In [ ]:
# Create DB Instance

!gcloud sql instances create {DB_INSTANCE_NAME} \
    --database-version={DB_VERSION} \
    --cpu={CPU} \
    --memory={MEMORY} \
    --region={REGION} \
    --edition={EDITION} \
    --root-password={ROOT_PASS}


Creating Cloud SQL instance for POSTGRES_17...done.                            
Created [https://sqladmin.googleapis.com/sql/v1beta4/projects/ari-dev-463216/instances/jerry-v1-db-instance].
NAME                  DATABASE_VERSION  LOCATION       TIER              PRIMARY_ADDRESS  PRIVATE_ADDRESS  STATUS
jerry-v1-db-instance  POSTGRES_17       us-central1-c  db-custom-1-4096  34.59.111.24     -                RUNNABLE


In [ ]:
# Delete DB Instance
# NOTE: this command requires a user input which can't be done in this file
#       the `-q` flag overides it
!gcloud sql instances delete {DB_INSTANCE_NAME} -q

Deleting Cloud SQL instance...done.                                            
Deleted [https://sqladmin.googleapis.com/sql/v1beta4/projects/ari-dev-463216/instances/jerry-v1-db-instance].


In [ ]:
# List DB Instance

!gcloud sql instances list --filter="name~{DB_INSTANCE_NAME}"

NAME                  DATABASE_VERSION  LOCATION       TIER              PRIMARY_ADDRESS  PRIVATE_ADDRESS  STATUS
jerry-v1-db-instance  POSTGRES_17       us-central1-c  db-custom-1-4096  34.59.111.24     -                RUNNABLE


In [37]:
# Create DB
!gcloud sql databases create {DB_NAME} -i={DB_INSTANCE_NAME}

Creating Cloud SQL database...done.                                            
Created database [jerry_v1_extracted_db].
instance: jerry-v1-db-instance
name: jerry_v1_extracted_db
project: ari-dev-463216


In [34]:
# Delete DB
# NOTE: this command requires a user input which can't be done in this file
#       the `-q` flag overides it
!gcloud sql databases delete {DB_NAME} -i={DB_INSTANCE_NAME} -q

Deleting Cloud SQL database...done.                                            
Deleted database [jerry_v1_extracted_db].


In [38]:
# List DB
!gcloud sql databases list -i={DB_INSTANCE_NAME}

NAME                   CHARSET  COLLATION
postgres               UTF8     en_US.UTF8
jerry_v1_extracted_db  UTF8     en_US.UTF8


In [39]:
# Create DB Tables
from google.cloud.sql.connector import Connector

CREATE_TABLE_QUERY = """
    CREATE TABLE IF NOT EXISTS orders (
        order_id BIGINT PRIMARY KEY,
        client_name TEXT NOT NULL,
        client_address TEXT NOT NULL,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );
    """

connector = Connector()
conn = connector.connect(
    CONNECTION_NAME,
    PG_DRIVER,
    user=USER,
    password=PASSWORD,
    db=DB_NAME
)

try:
    cur = conn.cursor()
    cur.execute(CREATE_TABLE_QUERY)
    conn.commit()
    print("Table created / already exists")
except Exception as e:
    print(f"Query Failed\nERROR: {e}")

except Exception as e:
    print(f"Connection Failed\nERROR: {e}")
finally:
    if cur is not None:
        cur.close()
    if conn is not None:
        conn.close()

Table created / already exists


In [ ]:
# List all folders in bucket
!gcloud storage folders list {BUCKET_BASE_URL} --format="value(name)"

failed/
processing/
succeded/
to_process/
